# Title: Scrapping Data for Top 10 Property Stock (Bursa Anouncement, Stock Historical Price Data, KLSE Historical Price Data, Quarterly Report Data, Property Stock Market Cap Data


# Import all the relevent libraries
1) Request to get url information by HTTP requests in Python <br>
2) Beautifulsoup to pull data out of HTML <br>
3) Panda to handle data,create dataframe and save as csv <br>
4) Panda Data reader to extract Historical Stock data from Yahoo Finance with Yahoo API <br>
5) Selenium use computer generated software with autoscrolling to extract data <br>
6) Time to set duration to wait for the page to load in selenium <br>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
#GOD SET WIDTH TO MAX
from pandas_datareader import data
## To extract historical price data
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


C:\Users\dsu.jianwei\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.
C:\Users\dsu.jianwei\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


# To extract the Market Cap data of the Property Stock
Used to determine the top 10 property stock

In [2]:
property_listurl = "https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=13"
## Property stocks url

In [3]:
get_prop = requests.get(property_listurl)
propsoup = BeautifulSoup(get_prop.text,"html.parser")
propcomp=propsoup.find("div",{"id":"companyInfo"}).find_all("a")
prop_marketcap = propsoup.find("div",{"id":"companyInfo"}).find_all("td")
## Collect property stock market cap data
# propsoup.find_all("a", href=lambda href: href and "https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=" in href)

In [4]:
def unit_price(x):
    if x[-1]=='b':
        return float(x[:-1])*1000000000
    elif x[-1]=='m':
        return float(x[:-1])*1000000
#tips to improve use dictionary
## Convert market cap millions & billions for sorting

In [5]:
comp_marketcap=[]
comp_lastprice=[]
comp_PE = []
comp_DY = []
comp_roe = []
count=1
for i in prop_marketcap:
    if count%8==4:
        comp_marketcap.append(i.string)
    if count%8==5:
        comp_lastprice.append(i.string)
    if count%8==6:
        comp_PE.append(i.string)
    if count%8==7:
        comp_DY.append(i.string)
    if count%8==0:
        comp_roe.append(i.string)
    count+=1
## Compile each item (marketcap,price,pe,dy,roe) in to list

In [9]:
comp_name=[]
comp_code=[]
comp_code_kl=[]
comp_url=[]
for i in propcomp:
    x=i.string.split(" ")
    comp_name.append(x[0])
    comp_code_kl.append(x[1][1:5]+'.KL')
    ### Add KL tag into the comp code for easier data extraction
    comp_code.append(x[1][1:5])
    comp_url.append("https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode="+x[1][1:5])
property_details ={'Company Name':comp_name,'Company Code':comp_code,'Company Code KL':comp_code_kl,'URL':comp_url,'Market Cap':comp_marketcap,'Last Price':comp_lastprice,'Price/Earning':comp_PE,'Divident Yield':comp_DY,'Return of Equity':comp_roe}
df_prop_detail = pd.DataFrame(property_details)
df_prop_detail['Market Cap Float']=df_prop_detail['Market Cap'].apply(unit_price)
##Using the function to convert mil & bil to sort later
top10market_cap=df_prop_detail.sort_values(by=['Market Cap Float'],ascending=False).head(10)
allmarket_cap=df_prop_detail.sort_values(by=['Market Cap Float'],ascending=False)
top10market_cap
### Generate the market cap dataframe

,Company Name,Company Code,Company Code KL,URL,Market Cap,Last Price,Price/Earning,Divident Yield,Return of Equity,Market Cap Float
35,IOIPG,5249,5249.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=5249,5.341b,0.97,7.96,3.09,3.55,5.341000e+09
77,SIMEPROP,5288,5288.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=5288,4.115b,0.61,6.88,4.96,6.15,4.115000e+09
80,SPSETIA,8664,8664.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=8664,3.679b,0.91,9.92,9.40,3.04,3.679000e+09
93,UOADEV,5200,5200.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=5200,3.639b,1.85,9.11,7.57,7.87,3.639000e+09
34,IGBB,5606,5606.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=5606,2.799b,3.15,13.41,0.95,4.43,2.799000e+09
59,MRCB,1651,1651.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=1651,2.007b,0.46,84.26,2.20,0.50,2.007000e+09
92,UEMS,5148,5148.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=5148,1.883b,0.42,8.42,0.00,3.06,1.883000e+09
64,OSK,5053,5053.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=5053,1.676b,0.80,4.07,6.25,8.16,1.676000e+09
50,MATRIX,5236,5236.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=5236,1.440b,1.75,5.89,7.29,15.55,1.440000e+09
96,YNHPROP,3158,3158.KL,https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode=3158,1.381b,2.61,32.75,0.00,3.62,1.381000e+09


In [10]:
top10market_cap.to_csv("Top 10 Property Stock Market Cap.csv")

# To extract the 10 years bursa announcement data for the top 10 property stock

In [7]:
bursa_url = "https://www.bursamalaysia.com/market_information/announcements/company_announcement?keyword=&cat=&sub_type=&company=5288&mkt=&alph=&sec=&subsec=&dt_ht=&dt_lt=&page=1"
bursa = requests.get(bursa_url).text
bursa_soup = BeautifulSoup(bursa,"html.parser")
bursa_last=bursa_soup.find("thead").find_all("th")
bursa_col = [i.string for i in bursa_last]
bursa_col

['No.', 'Announcement Date', 'Company Name', 'Title']

In [27]:
def get_bursa_page(code):
    url="https://www.bursamalaysia.com/market_information/announcements/company_announcement?keyword=&cat=&sub_type=&company="+str(code)+"&mkt=&alph=&sec=&subsec=&dt_ht=&dt_lt=&page=1"
    bursa=requests.get(url).text
    bursa_soup=BeautifulSoup(bursa,"html.parser")
    bursa_last=bursa_soup.find_all("a",{"data-dt-idx":"1"})
    bursa_last=bursa_last[-1].string
    bursa_output = []
    bursa_anounce=[]
    bursa_final_output=[]
    for j in range(1,int(bursa_last)+1):
        count=1
        new_url = "https://www.bursamalaysia.com/market_information/announcements/company_announcement?keyword=&cat=&sub_type=&company="+str(code)+"&mkt=&alph=&sec=&subsec=&dt_ht=&dt_lt=&page="+str(j)
        bursa=requests.get(new_url).text
        bursa_soup=BeautifulSoup(bursa,"html.parser")
        for i in bursa_soup.find("tbody").find_all("td"):
            if i.string == None:
                bursa_anounce.append(i.find("a").string.strip("\n").strip())
                count+=1
            else:
                bursa_anounce.append(i.string.strip("\n").strip())
                count+=1
            if count>4:
                bursa_output.append(bursa_anounce)
                bursa_anounce=[]
                count=1
        bursa_final_output=bursa_final_output+bursa_output
    temp = pd.DataFrame(bursa_final_output,columns=bursa_col)[['Announcement Date', 'Company Name', 'Title']]
    temp['Company Code']= str(code)
    return temp

In [90]:
first_row = [[0,0,0,0]]
first_col = ['Announcement Date', 'Company Name', 'Title','Company Code']
temp_output=pd.DataFrame(first_row,columns=first_col)
for i in top10market_cap['Company Code']:
    temp_output=temp_output.append(get_bursa_page(i))
    print("Done")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [91]:
temp_output=temp_output[1:]
temp_output.to_csv("Bursa_Announcement.csv")

In [92]:
temp_output

,Announcement Date,Company Name,Title,Company Code
0,12 Mar 2020,IOI PROPERTIES GROUP BERHAD,DEALINGS IN LISTED SECURITIES (CHAPTER 14 OF LISTING REQUIREMENTS) : DEALINGS OUTSIDE CLOSED PERIOD,5249
1,12 Mar 2020,IOI PROPERTIES GROUP BERHAD,Changes in Director's Interest (Section 219 of CA 2016) - MISS LEE YOKE HAR,5249
2,12 Mar 2020,IOI PROPERTIES GROUP BERHAD,Changes in Director's Interest (Section 219 of CA 2016) - DATO LEE YEOW CHOR,5249
3,12 Mar 2020,IOI PROPERTIES GROUP BERHAD,Changes in Director's Interest (Section 219 of CA 2016) - MR LEE YEOW SENG,5249
4,12 Mar 2020,IOI PROPERTIES GROUP BERHAD,Changes in Sub. S-hldr's Int (Section 138 of CA 2016) - VERTICAL CAPACITY SDN BHD,5249
...,...,...,...,...
37814,10 Feb 2000,SANDA INDUSTRIES BERHAD,"SANDA INDUSTRIES BERHAD (""SANDA"" OR ""COMPANY"") NON-RENOUNCEABLE OFFER FOR SALE OF 33,000,000 ORDINARY SHARES OF RM1.00 EACH IN SANDA (""OFFER SHARES""), (""OFFER FOR SALE"")",8206
37815,10 Feb 2000,SANDA INDUSTRIES BERHAD,"SANDA-Non-Renounceable Offer For Sale Of 33,000,000 Ordinary Shares Of RM1.00 Each In Sanda By The Substantial Shareholders Of Focal Aims Properties Sdn Bhd (""FAPSB"") To The Existing Shareholders Of Sanda On The Basis Of Two (2) Shares For Every One (1) Share Held In Sanda At An Offer Price Of RM1.00 Per Share (""Non-Renounceable Offer For Sale"")",8206
37816,09 Feb 2000,SANDA INDUSTRIES BERHAD,UNUSUAL MARKET ACTION,8206
37817,31 Jan 2000,SANDA INDUSTRIES BERHAD,SANDA INDUSTRIES BERHAD (SANDA) - CEASATION OF OPERATION OF SUBSIDIARY,8206


# To extract the 10 Years historical data of Top 10 property stock using Yahoo Finance Api

In [11]:
for i in top10market_cap['Company Code KL'][:1]:
    # Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
    tickers = [str(i)]

    # We would like all available data from 01/01/2000 until 12/31/2016.
    start_date = '2010-01-01'
    end_date = '2020-01-01'

    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    panel_data = data.DataReader(str(i), 'yahoo', start_date, end_date)
    historicalprice=pd.DataFrame(panel_data).reset_index()
    historicalprice['Company Code KL']= str(i)
###### Extract top 10 historical price for past 10 years using api

In [12]:
for i in top10market_cap['Company Code KL'][1:10]:
    # Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
    tickers = [str(i)]

    # We would like all available data from 01/01/2000 until 12/31/2016.
    start_date = '2010-01-01'
    end_date = '2020-01-01'

    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    panel_data = data.DataReader(str(i), 'yahoo', start_date, end_date)
    sub_hist = pd.DataFrame(panel_data).reset_index()
    sub_hist['Company Code KL']= str(i)
    historicalprice=historicalprice.append(sub_hist)
###### Extract top 10 historical price for past 10 years using api
historicalprice

,Date,High,Low,Open,Close,Volume,Adj Close,Company Code KL
0,2014-01-15,3.21460,2.71796,2.89855,2.79923,73229619.0,2.323783,5249.KL
1,2014-01-16,2.81729,2.60057,2.79923,2.63669,49003730.0,2.188851,5249.KL
2,2014-01-20,2.75408,2.68184,2.71796,2.73602,19515133.0,2.271310,5249.KL
3,2014-01-21,2.70893,2.68184,2.69990,2.69990,13927056.0,2.241324,5249.KL
4,2014-01-22,2.71796,2.62766,2.69990,2.63669,25070650.0,2.188851,5249.KL
...,...,...,...,...,...,...,...,...
1987,2018-01-23,1.40000,1.38000,1.38000,1.40000,247700.0,1.400000,3158.KL
1988,2018-01-24,1.40000,1.38000,1.38000,1.40000,44200.0,1.400000,3158.KL
1989,2018-01-26,1.41000,1.39000,1.39000,1.41000,374800.0,1.410000,3158.KL
1990,2018-01-29,1.39000,1.38000,1.39000,1.39000,347400.0,1.390000,3158.KL


In [14]:
historicalprice.to_csv("Historical Price.csv")

# To extract the 10 years KLSE data that will be used as benchmark

In [31]:
klse_url="https://finance.yahoo.com/quote/%5EKLSE%3FP%3D%5EKLSE/history?period1=754876800&period2=1583798400&interval=1d&filter=history&frequency=1d"
get_klse = requests.get(klse_url).text
klse_input=BeautifulSoup(get_klse,"html.parser")
klse_col_name = [i.string for i in klse_input.find("table",{"class":"W(100%) M(0)"}).find_all("th")]


In [32]:
driver = webdriver.Chrome('C:/Users/dsu.jianwei/Desktop/chromedriver')
driver.get("https://finance.yahoo.com/quote/%5EKLSE%3FP%3D%5EKLSE/history?period1=1262304000&period2=1577836800&interval=1d&filter=history&frequency=1d")
html = driver.find_element_by_tag_name('html')


# temp = ""
# while temp != "Yes":
#     if 'Jan 04, 2010' in init_data[-7]:
#         temp = "Yes"
#     else:
# while 'Jan 04, 2010' not in init_data[-20:]:
#     print(len(init_data[-20:]))
for i in range(30):
    html.send_keys(Keys.END)
    SCROLL_PAUSE_TIME = 0.5
    time.sleep(SCROLL_PAUSE_TIME)
#     init_data = driver.find_elements_by_tag_name('td')
the_html = driver.find_element_by_tag_name("tbody").text
print("DONE")

    

DONE


In [18]:
data_first= the_html.split("\n")

In [19]:
def clean_data(i):
    temp=[]
    first = i[0]+ i[1] + i[2]
    temp.append(first)
    temp=temp+i[3:]
    return temp

In [20]:
data_output=[]
for i in data_first:
    temp_x=i.split(" ")
    data_output.append(clean_data(temp_x))
df_klse= pd.DataFrame(data_output,columns=klse_col_name)


In [21]:
df_klse

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Dec31,2019","1,609.37","1,610.69","1,588.76","1,588.76","1,588.76","104,601,300"
1,"Dec30,2019","1,609.45","1,617.43","1,608.76","1,615.67","1,615.67","82,149,800"
2,"Dec27,2019","1,599.93","1,610.91","1,598.48","1,610.61","1,610.61","63,765,200"
3,"Dec26,2019","1,604.68","1,607.12","1,602.66","1,603.55","1,603.55","22,687,500"
4,"Dec24,2019","1,613.83","1,613.87","1,602.10","1,604.23","1,604.23","34,847,100"
...,...,...,...,...,...,...,...
2465,"Jan08,2010","1,294.93","1,295.51","1,290.86","1,292.98","1,292.98","74,587,200"
2466,"Jan07,2010","1,293.69","1,299.70","1,290.36","1,291.42","1,291.42","115,024,400"
2467,"Jan06,2010","1,288.86","1,296.44","1,288.02","1,293.17","1,293.17","117,740,300"
2468,"Jan05,2010","1,278.26","1,290.55","1,278.26","1,288.24","1,288.24","136,646,600"


In [22]:
df_klse.to_csv('KLSE_Historical_data.csv')

# To extract Quarterly Report Data for the Top 10 property stock

In [23]:
top_10_compcode=[i for i in top10market_cap['Company Code']]
all_compcode = [i for i in allmarket_cap['Company Code']]
## list for top 10 comp code & full list of prop comp code

In [24]:
def scrap_break_header(input_col):
    for br in input_col.findAll('br'):
        next_s = br.nextSibling
        prev_s = br.previousSibling
        return str(prev_s)+' '+str(next_s)
## For quarter report column name that consist of <br> e.g. Finance <br> Year convert to Finance Year

In [25]:
def remove_unwanted(soup_input):
    count=0
    colspan11=soup_input.find(id='MainContent_gvReport').find_all("td",{'colspan':"11"})
    for i in soup_input.find(id='MainContent_gvReport').find_all('td'):
        if str(i) in str(colspan11):
            count+=1
    return -count
## Remove last part of the data table that is irrelevant

In [26]:
### Function to extract page >2 in the quarter report data
def other_pages(data_input):
    table_values = data_input.find(id='MainContent_gvReport').find_all('td')
    table_values_1=table_values[:remove_unwanted(data_input)]
    def check_label(x):
        label_img_list=[]
        count=-1
        for i in x:
            count+=1
            if ("label" in str(i)):
                label_img_list.append(count)
        return(label_img_list)
    index_label_img=check_label(table_values_1)
    def regroup(x,data):
        if x==0:
            col0.append(data)    
        if x==1:
            col1.append(data)
        if x==2:
            col2.append(data)
        if x==3:
            col3.append(data)
        if x==4:
            col4.append(data)
        if x==5:
            col5.append(data)
        if x==6:
            col6.append(data)
        if x==7:
            col7.append(data)
        if x==8:
            col8.append(data)
        if x==9:
            col9.append(data)
    #Processing and separating the data
    count=0
    # image_label_output=[]
    #Data for each column
    col0=[]
    col1=[]
    col2=[]
    col3=[]
    col4=[]
    col5=[]
    col6=[]
    col7=[]
    col8=[]
    col9=[]
    col10=[]
    for i in table_values_1:
        if table_values_1.index(i) in index_label_img:
            col10.append(i.find("label").string)
        else:
            if count<120:
                regroup(count%10,i.string)
                count+=1

    col_list= [col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10]
    df = pd.DataFrame(col_list)
    df_transposed=df.transpose()
    df_transposed.columns=table_col_list
    return df_transposed

In [27]:
## Function to get the full data page 1 to max page
def get_comp_quarter_data(code):
    ## URL for the company code 
    url = "https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode="+str(code)
    getreq = requests.get(url)
    firstsoup = BeautifulSoup(getreq.text,"html.parser")
    header=code
    ## find the viewstate & event validation that is required for extracting POST REQUEST FOR page >2
    VIEWSTATE = firstsoup.find('input',{'id':'__VIEWSTATE'}).get('value')
    EVENTVALIDATION = firstsoup.find('input',{'id':'__EVENTVALIDATION'}).get('value')
    ## function to get the details of the other pages (page>2)
    def get_each_page(input_page):
        data = {
            "__EVENTTARGET": "ctl00$MainContent$gvReport",
            "__EVENTARGUMENT": input_page,
            "__EVENTVALIDATION":EVENTVALIDATION,
            "__VIEWSTATE":VIEWSTATE,
               }
        r = requests.post(url,data=data)
        soup=BeautifulSoup(r.text,"html.parser")
        return soup
    ##First soup = data from first page only
    table_col_name = firstsoup.find(id='MainContent_gvReport').find_all('th')
    table_col_list=[]
    for i in table_col_name:
        if i.string==None:
            table_col_list.append(scrap_break_header(i))
        else:
            table_col_list.append(i.string)

            ###########Get first page
    table_values = firstsoup.find(id='MainContent_gvReport').find_all('td')
    ########## check how many pages & what is the last page 
    last_page=(table_values[-1].string)
#     print(last_page)
    table_values_1=table_values[:remove_unwanted(firstsoup)]
    ### remove unwanted data
    
    def check_label(x):
        label_img_list=[]
        count=-1
        for i in x:
            count+=1
            if ("label" in str(i)):
                label_img_list.append(count)
        return(label_img_list)
    ### to extract last column which is in different format 
    index_label_img=check_label(table_values)
    def regroup(x,data):
        if x==0:
            col0.append(data)    
        if x==1:
            col1.append(data)
        if x==2:
            col2.append(data)
        if x==3:
            col3.append(data)
        if x==4:
            col4.append(data)
        if x==5:
            col5.append(data)
        if x==6:
            col6.append(data)
        if x==7:
            col7.append(data)
        if x==8:
            col8.append(data)
        if x==9:
            col9.append(data)
    ## each sequence represent their respective columns
    #Processing and separating the data
    count=0
    # image_label_output=[]
    #Data for each column
    col0=[]
    col1=[]
    col2=[]
    col3=[]
    col4=[]
    col5=[]
    col6=[]
    col7=[]
    col8=[]
    col9=[]
    col10=[]
    ## Col 10 is the last column with label attribute
    for i in table_values:
        if table_values.index(i) in index_label_img:
            col10.append(i.find("label").string)
        else:
            if count<120:
                regroup(count%10,i.string)
                count+=1
### use mod 10 so for every 10 will go to different columns
    col_list= [col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10]
    ## put each columns in to a list
    df = pd.DataFrame(col_list)
    ## put it in data frame
    df_transposed=df.transpose()
    ### transposed the data 
    df_transposed.columns=table_col_list
    first_page=df_transposed
#     print(first_page)
####################
########Get other pages (page>2)
    def other_pages(data_input):
        table_values = data_input.find(id='MainContent_gvReport').find_all('td')
        table_values_1=table_values[:remove_unwanted(data_input)]
        def check_label(x):
            label_img_list=[]
            count=-1
            for i in x:
                count+=1
                if ("label" in str(i)):
                    label_img_list.append(count)
            return(label_img_list)
        index_label_img=check_label(table_values_1)
        def regroup(x,data):
            if x==0:
                col0.append(data)    
            if x==1:
                col1.append(data)
            if x==2:
                col2.append(data)
            if x==3:
                col3.append(data)
            if x==4:
                col4.append(data)
            if x==5:
                col5.append(data)
            if x==6:
                col6.append(data)
            if x==7:
                col7.append(data)
            if x==8:
                col8.append(data)
            if x==9:
                col9.append(data)
        #Processing and separating the data
        count=0
        # image_label_output=[]
        #Data for each column
        col0=[]
        col1=[]
        col2=[]
        col3=[]
        col4=[]
        col5=[]
        col6=[]
        col7=[]
        col8=[]
        col9=[]
        col10=[]
        for i in table_values_1:
            if table_values_1.index(i) in index_label_img:
                col10.append(i.find("label").string)
            else:
                if count<120:
                    regroup(count%10,i.string)
                    count+=1

        col_list= [col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10]
        df = pd.DataFrame(col_list)
        df_transposed=df.transpose()
        df_transposed.columns=table_col_list
        return df_transposed
    ####### To append the other pages to the first page extracted
    if last_page!=None:
        for i in range(2,int(last_page)+1):
#             print(i)
            first_page=first_page.append(other_pages(get_each_page('Page$'+str(i))))
        else:
            first_page
    first_page=first_page.reset_index(drop="True")
    first_page['Company Code']=str(code)
    ## add a new column to indicate the company's code
    return first_page


In [29]:
# print(top_10_compcode)
# print(all_compcode)
ten_prop_historical = get_comp_quarter_data(top_10_compcode[0])
for i in range(1,len(top_10_compcode)):
    ten_prop_historical=ten_prop_historical.append(get_comp_quarter_data(top_10_compcode[i]))
ten_prop_historical
#Checker whether fail

,Date,Financial Year,No.,Financial Quarter,"Revenue (RM,000)","PBT (RM,000)","Net Profit (RM,000)",EPS (Cent),Dividend (Cent),NTA (RM),YoY (%),Company Code
0,25 Feb 2020,30 Jun 2020,2,31 Dec 2019,"564,129","286,301","199,749",3.63,0.00,3.430,7%,5249
1,25 Nov 2019,30 Jun 2020,1,30 Sep 2019,"540,315","255,649","136,636",2.48,0.00,3.400,22%,5249
2,29 Aug 2019,30 Jun 2019,4,30 Jun 2019,"497,791","227,892","139,768",2.54,3.00,3.420,47%,5249
3,27 May 2019,30 Jun 2019,3,31 Mar 2019,"487,739","300,902","194,700",3.54,0.00,3.380,17%,5249
4,26 Feb 2019,30 Jun 2019,2,31 Dec 2018,"666,150","355,012","214,864",3.90,0.00,3.360,97%,5249
5,23 Nov 2018,30 Jun 2019,1,30 Sep 2018,"560,062","202,154","111,958",2.03,0.00,3.320,54%,5249
6,27 Aug 2018,30 Jun 2018,4,30 Jun 2018,"673,975","306,261","264,993",4.81,5.00,3.330,21%,5249
7,18 May 2018,30 Jun 2018,3,31 Mar 2018,"541,212","221,734","166,647",3.03,0.00,3.270,38%,5249
8,23 Feb 2018,30 Jun 2018,2,31 Dec 2017,"707,444","189,412","109,139",1.98,0.00,3.280,60%,5249
9,20 Nov 2017,30 Jun 2018,1,30 Sep 2017,"869,979","354,125","242,852",4.41,0.00,3.290,28%,5249


In [30]:
ten_prop_historical.to_csv("Scraped_data_property.csv")